1. Посмотреть распределение аллофонов:

1.0. Определить группы контекстов  
1.1. Прочитать файлы  
1.2. Найти аллофоны (B1)  
1.2.1. Сохранить аллофон с соседями  

In [ ]:
left_vowel_contexts = {
    
}

In [1]:
from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
level_codes = [2 ** i for i in range(len(levels))]
code_to_level = {i: j for i, j in zip(level_codes, levels)}
level_to_code = {j: i for i, j in zip(level_codes, levels)}

In [2]:
def read_seg(filename: str, encoding: str = "utf-8-sig") -> tuple[dict, list[dict]]:
    with open(filename, encoding=encoding) as f:
        lines = [line.strip() for line in f.readlines()]

    # найдём границы секций в списке строк:
    header_start = lines.index("[PARAMETERS]") + 1
    data_start = lines.index("[LABELS]") + 1

    # прочитаем параметры
    params = {}
    for line in lines[header_start:data_start - 1]:
        key, value = line.split("=")
        params[key] = int(value)

    # прочитаем метки
    labels = []
    for line in lines[data_start:]:
        # если в строке нет запятых, значит, это не метка и метки закончились
        if line.count(",") < 2:
            break
        pos, level, name = line.split(",", maxsplit=2)
        label = {
            "position": int(pos) // params["BYTE_PER_SAMPLE"] // params["N_CHANNEL"],
            "level": code_to_level[int(level)],
            "name": name
        }
        labels.append(label)
    return params, labels

Нам понадобится файл cta_seg.zip 

In [ ]:
!unzip -q cta_seg.zip

In [5]:
import glob
files_B1 = glob.glob("cta_seg/*.seg_B1", recursive=True)

In [10]:
from collections import Counter

In [ ]:
vowels = "i e a o u y".split()

In [ ]:
permissible_allophones = [
    ('a1', 3748),
    ('a0', 3178),
    ('t', 3154),
    ('o0', 3128),
    ('i1', 3063),
    ('n', 2948),
    ('y4', 2699),
    ('i4', 2682),
    ('l', 2149),
    ('k', 2135),
    ('e0', 2111),
    ('v', 2035),
    ('r', 2017),
    ('s', 2015),
    ("n'", 1800),
    ('d', 1726),
    ('m', 1664),
    ('a4', 1614),
    ('p', 1612),
    ('e4', 1468),
    ("l'", 1432),
    ('i0', 1311),
    ('y1', 1272),
    ("s'", 1201),
    ("t'", 1144),
    ("r'", 1079),
    ('j', 1063),
    ('z', 1017),
    ('sh', 914),
    ('ch', 906),
    ('g', 859),
    ('b', 842),
    ('u0', 840),
    ('u1', 807),
    ('f', 785),
    ("m'", 767),
    ('u4', 750),
    ('zh', 737),
    ("v'", 663),
    ("d'", 648),
    ('y0', 636),
    ('h', 542),
    ('e1', 484),
    ('c', 368),
    ('a2', 365),
    ("k'", 307),
    ("b'", 260),
    ('o4', 243),
    ('sc', 233),
    ("p'", 228),
    ("z'", 178),
    ('o1', 147),
    ("g'", 95),
    ("f'", 78),
    ('H', 28),
    ("h'", 25),
    ('ch_', 16),
    ("zh'", 14),
    ('~', 5),
    ('CH', 3),
    ('C', 3),
]

In [ ]:
consonants = [i[0] for i in permissible_allophones if i[0][0] not in vowels]

In [ ]:
class2sound_vl = {
    "labials": "f v p b".split(),
    "labials_palat": "f' v' p' b'".split(),
    "dentals": "t d s z c".split(),
    "dentals_palat": "t' d' s' z' ch sc r' zh'".split(),
    "postalveolars": "sh zh ch_".split(),
    "velars": "k g h".split(),
    "velars_palat": "k' g' h'".split(),
    "start": [""]
}
class2sound_vl.update({i: [i] for i in "m m' n n' l l' r j a o u e i y".split()})

In [ ]:
sound2class_vl = {j: i for i in class2sound_vl for j in class2sound_vl[i]}

In [ ]:
class2sound_vr = {
    "labials": "p b m f v".split(),
    "dentals": "t d s z c C n r".split(),
    "postalveolars": "sh zh ch_".split(),
    "velars": "k g h H".split(),
    "palat": "f' v' p' b' t' d' s' z' ch sc r' zh' k' g' h' m' n' l' r'".split(),
    "end": [""]
}
class2sound_vr.update({i: [i] for i in "l j a o u e i y".split()})

In [ ]:
sound2class_vr = {j: i for i in class2sound_vr for j in class2sound_vr[i]}

In [ ]:
class2sound_cl = {
    "vowels": vowels,
    "consonants": consonants,
    "start": [""]
}

In [ ]:
sound2class_cl = {j: i for i in class2sound_cl for j in class2sound_cl[i]}

In [ ]:
class2sound_cr = {
    "rounded": "o u".split(),
    "unrounded": "i e a y".split(),
    "voiceless": "t k s p s' t' sh ch f h c k' sc p' f' h' ch_".split(),
    "sonorants": "n l v r n' m l' r' j m' v'".split(),
    "voiced": "d z g b zh d' b' z' g' H CH C".split(),
    "end": [""]
}

In [ ]:
sound2class_cr = {j: i for i in class2sound_cr for j in class2sound_cr[i]}

In [ ]:
parasm, labels = read_seg("cta_seg_utf8/cta0001.seg_B1")

In [ ]:
import re

In [ ]:
labels = [{"position": 0, "name": ""}] + labels
labels_context = []

for l_1, l_2, l_3 in zip(labels, labels[1:], labels[2:]):
    if l_2["name"].startswith(tuple(vowels)):
        cur_class = "vowel"
    elif l_2["name"] in consonants:
        cur_class = "consonant"
    else:
        continue
    left_context = re.sub("\d", "", l_1["name"])
    sound2class_left = sound2class_cl if cur_class == "consonant" else sound2class_vl
    sound2class_right = sound2class_cr if cur_class == "consonant" else sound2class_vr
    left_class = sound2class_left.get(left_context, "???")
    right_context = re.sub("\d", "", l_3["name"])
    right_class = sound2class_right.get(right_context, "???")
    labels_context.append((left_class, l_2["name"], right_class))

In [15]:
allophones = Counter()
for file in files_B1:
    params, labels = read_seg(file)
    phones = [l["name"] for l in labels if l["name"]]
    if "zh'" in phones:
        print(file)
    allophones.update(phones)

d:/corpres/careva/cta\cta0001-0010\cta0005.seg_B1
d:/corpres/careva/cta\cta0191-0200\cta0200.seg_B1
d:/corpres/careva/cta\cta0331-0340\cta0338.seg_B1
d:/corpres/careva/cta\cta0371-0380\cta0378.seg_B1
d:/corpres/careva/cta\cta0661-0670\cta0663.seg_B1
d:/corpres/careva/cta\cta0731-0740\cta0736.seg_B1
d:/corpres/careva/cta\cta0771-0780\cta0779.seg_B1
d:/corpres/careva/cta\cta0781-0790\cta0784.seg_B1
d:/corpres/careva/cta\cta0911-0920\cta0920.seg_B1
d:/corpres/careva/cta\cta1001-1010\cta1003.seg_B1


In [12]:
allophones.most_common()

[('a1', 3748),
 ('a0', 3178),
 ('t', 3154),
 ('o0', 3128),
 ('i1', 3063),
 ('n', 2948),
 ('y4', 2699),
 ('i4', 2682),
 ('l', 2149),
 ('k', 2135),
 ('e0', 2111),
 ('v', 2035),
 ('r', 2017),
 ('s', 2015),
 ("n'", 1800),
 ('d', 1726),
 ('m', 1664),
 ('a4', 1614),
 ('p', 1612),
 ('e4', 1468),
 ("l'", 1432),
 ('i0', 1311),
 ('y1', 1272),
 ("s'", 1201),
 ("t'", 1144),
 ("r'", 1079),
 ('j', 1063),
 ('z', 1017),
 ('sh', 914),
 ('ch', 906),
 ('g', 859),
 ('b', 842),
 ('u0', 840),
 ('u1', 807),
 ('f', 785),
 ("m'", 767),
 ('u4', 750),
 ('zh', 737),
 ("v'", 663),
 ("d'", 648),
 ('y0', 636),
 ('h', 542),
 ('e1', 484),
 ('c', 368),
 ('a2', 365),
 ("k'", 307),
 ("b'", 260),
 ('o4', 243),
 ('sc', 233),
 ("p'", 228),
 ("z'", 178),
 ('o1', 147),
 ("g'", 95),
 ("f'", 78),
 ('H', 28),
 ("h'", 25),
 ('ch_', 16),
 ("zh'", 14),
 ('~', 5),
 ('CH', 3),
 ('C', 3),
 ('04', 2),
 ('i4 j i4', 1),
 (' n', 1),
 ('ie4', 1),
 (' r', 1)]